* The following code is modified based on the original torchcv project. 
* We are going to use PASACAL VOC12 as dataset.
* You could donwload VOC2012 
  train/validation: http://host.robots.ox.ac.uk/pascal/VOC/voc2012/#devkit
* test data: https://pjreddie.com/projects/pascal-voc-dataset-mirror/


In [1]:
import os
import random
import argparse
from PIL import Image
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

from __future__ import print_function
from dbpn import Net as DBPN
from dbpn import get_pair_set
from ssd import SSD
from ssd import build_ssd
from ssd.layers.modules import MultiBoxLoss
from ssd.data.config import voc
from ssd.data import detection_collate
from ssd.data import VOCAnnotationTransform, VOCDetection, BaseTransform, SRDetection

import sys; sys.argv=['']; del sys

In [2]:
# Arguments & settings
parser = argparse.ArgumentParser(description='PyTorch Super Resolution Detection Networks')
parser.add_argument('--upscale_factor', type=int, default=8, help="super resolution upscale factor")
parser.add_argument('--testBatchSize', type=int, default=1, help='testing batch size')
parser.add_argument('--batch_size', type=int, default=64, help='training batch size')
parser.add_argument('--threads', type=int, default=4, help='number of threads for data loading')
parser.add_argument('--seed', type=int, default=123, help='random seed to use. Default=123')
parser.add_argument('--gpu_mode', type=bool, default=True)
parser.add_argument('--gpus', default=1, type=float, help='number of gpu')
parser.add_argument('--test_dataset', type=str, default='VOC12-LR-X8-test')
parser.add_argument('--train_dataset', type=str, default='VOC12-LR-X8')
parser.add_argument('--sr_dataset', type=str, default='VOC12-SR-X8')
parser.add_argument('--hr_dataset', type=str, default='VOC12-HR')
parser.add_argument('--anno_path', type=str, default='Annotations')
parser.add_argument('--imSetpath', type=str, default='ImageSets')
parser.add_argument('--input_dir', type=str, default='./dataset')
parser.add_argument('--output', default='./dataset/results', help='Location to save checkpoint models')
parser.add_argument('--lr', type=float, default=1e-4, help='Learning Rate. Default=0.0001')
parser.add_argument('--nEpochs', type=int, default=20, help='number of epochs to fine tune net S over target loss')

opt = parser.parse_args()

gpus_list=range(opt.gpus)
print(opt)

cuda = opt.gpu_mode
if cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run without --cuda")

torch.manual_seed(opt.seed)
if cuda:
    torch.cuda.manual_seed(opt.seed)
    torch.cuda.empty_cache()


Namespace(anno_path='Annotations', batch_size=64, gpu_mode=True, gpus=1, hr_dataset='VOC12-HR', imSetpath='ImageSets', input_dir='./dataset', lr=0.0001, nEpochs=20, output='./dataset/results', seed=123, sr_dataset='VOC12-SR-X8', testBatchSize=1, test_dataset='VOC12-LR-X8-test', threads=4, train_dataset='VOC12-LR-X8', upscale_factor=8)


In [ ]:
print(' YOU Donot need to run this ===> Loading some datasets')

lr_path = os.path.join(opt.input_dir, opt.train_dataset)
hr_path = os.path.join(opt.input_dir, opt.hr_dataset)

fine_train_set = get_pair_set(lr_path, hr_path)
train_data_loader = DataLoader(dataset=fine_train_set, num_workers=opt.threads, \
                               batch_size=opt.testBatchSize, shuffle=False)


In [3]:
print('===> Loading net S fine-tune training datasets')

sd_dataset = SRDetection(root='./dataset', image_sets='trainval.txt',
                         target_transform = VOCAnnotationTransform())


sd_data_loader = DataLoader(sd_dataset, opt.batch_size,num_workers=opt.threads,
                            shuffle=False, collate_fn=detection_collate)


===> Loading net S fine-tune training datasets


In [4]:
# TODO:
# 1. define Loss
# 2. come up with train, test utility functions

# VOC0712 dataset mean
MEANS = (104, 117, 123)
mean = np.array(MEANS, dtype=np.float32)

class DBPN2SSD(nn.Module):
    
    def __init__(self, s_model_name, d_model_name, d_frozen):
        super(DBPN2SSD, self).__init__()
        self.supervis = DBPN(num_channels=3, base_filter=64, feat=256, num_stages=7, scale_factor=4)
        if os.path.exists(s_model_name):
            self.supervis = torch.nn.DataParallel(self.supervis)
            self.supervis.load_state_dict(torch.load(s_model_name, map_location=lambda storage, loc: storage))

        # self.detector = SSD(), setup ssd as 'train' mode for gradient flow
        # later at test/eval situation, we will overwrite it's mode to 'test'
        self.detector = build_ssd('train', 300, 21)
        if os.path.exists(d_model_name):
            self.detector.load_state_dict(torch.load(d_model_name, map_location=lambda storage, loc: storage))
        if d_frozen:
            for param in self.detector.parameters():
                param.requires_grad = False
        
    def forward(self, x):
        superx = self.supervis(x)
        # current detector: SSD300, so assume superx: 300x300!
        superx -= mean
        superx = superx.astype(np.float32)
        detect = self.detector(superx)
        return (superx, detect)
        
    

In [ ]:
def simple_test():
    SDnet = DBPN2SSD('dbpn/models/DBPN_x4.pth', 'ssd/weights/ssd300_mAP_77.43_v2.pth', True)
    net = SDnet
    
    if cuda:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
        net = torch.nn.DataParallel(net)
        cudnn.benchmark = True
        net = net.cuda()

    else:
        torch.set_default_tensor_type('torch.FloatTensor')


    
simple_test()

In [5]:
alpha = 1.0
beta = 1.0


def train():
    SDnet = DBPN2SSD('dbpn/models/DBPN_x4.pth', 'ssd/weights/ssd300_mAP_77.43_v2.pth', True)
    net = SDnet
    
    if cuda:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
        net.supervis = torch.nn.DataParallel(net.supervis)
        cudnn.benchmark = True
        net = net.cuda()

    # we can be Specific to what part parameters to optimize
    optimizer = optim.Adam(net.supervis.parameters(), lr=opt.lr, betas=(0.9, 0.999), eps=1e-8)
    epoch_loss = 0
    
    # set model to training mode
    net.train()
    
    # ssd loss function
    criterion_ssd = MultiBoxLoss(voc['num_classes'], 0.5, True, 0, True, 3, 0.5, False, cuda)
    
    # net SR loss function, change to L2 later
    criterion_sr = nn.L1Loss()
    
    for iteration, batch in enumerate(sd_data_loader, 1):
        # input is LR image; target is pseudo (300x300) original HR image
        input, sr_target, det_target = Variable(batch[0]), Variable(batch[1]), variable(batch[2])
        if cuda:
            input = input.cuda(gpus_list[0])
            sr_target = sr_target.cuda(gpus_list[0])
            det_target = det_target.cuda(gpus_list[0])

        optimizer.zero_grad()
        t0 = time.time()
        
        sr_out,ssd_out = net(input)
        loss_sr = criterion_sr(sr_out, sr_target)
        
        # TODO: need code up detector's output - targets
        loss_l, loss_c = criterion_ssd(ssd_out, det_target)
        loss = loss_sr * alpha + beta *(loss_l + loss_c)
        
        t1 = time.time()
        epoch_loss += loss.data[0]
        loss.backward()
        optimizer.step()

        print("===> Epoch[{}]({}/{}): Loss: {:.4f} || Timer: {:.4f} sec.".format(epoch, \
                    iteration, len(sd_data_loader), loss.data[0], (t1 - t0)))

    print("===> Epoch {} Complete: Avg. Loss: {:.4f}".format(epoch, \
                    epoch_loss / len(sd_data_loader)))

In [6]:
train()

/home/x/cs231n/super-resolution-detection/dbpn/dbpn.py:47: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  torch.nn.init.kaiming_normal(m.weight)
/home/x/cs231n/super-resolution-detection/dbpn/dbpn.py:51: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  torch.nn.init.kaiming_normal(m.weight)
/home/x/cs231n/super-resolution-detection/ssd/ssd.py:34: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)
/home/x/cs231n/super-resolution-detection/ssd/layers/modules/l2norm.py:17: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.weight,self.gamma)


TypeError: Traceback (most recent call last):
  File "/home/x/anaconda3/envs/cs231n/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/x/cs231n/super-resolution-detection/ssd/data/__init__.py", line 26, in detection_collate
    targets.append(torch.FloatTensor(sample[1]))
TypeError: expected torch.FloatTensor (got torch.ByteTensor)


In [ ]:
torch.cuda.empty_cache()